In [1]:
import pandas as pd
import re
from keybert import KeyBERT
import yfinance as yf
import time
import random
import numpy as np

news_raw = pd.read_csv("C:/projectnasdaq/news_raw.csv", encoding='latin1')
nasdaq_stock = pd.read_csv("C:/projectnasdaq/nasdaq_stocks.csv", encoding='latin1')
rep_stock = pd.read_csv("C:/projectnasdaq/nasdaq_stocks_refine_total.csv", encoding='latin1')

news_raw = news_raw.head(1000)
news_raw = news_raw[['news_id','title','summary']]
nasdaq_stock = nasdaq_stock[['pk','symbol','name']]
rep_stock = rep_stock[['pk','symbol','name','name_a']]

# new_raw 데이터 전처리
news_raw['summary'] = news_raw['summary'].str.lower()
news_raw['summary'] = news_raw['summary'].apply(lambda x: re.sub('[^a-zA-Z\d&]', ' ', str(x)).strip())
self_stop_words = {'bloomberg'}
news_raw['summary'].replace(self_stop_words, '', regex=True, inplace=True)

# news_raw 데이터 토큰화
news_raw['tokenize'] = 0
news_raw['tokenize'] = news_raw['summary'].str.split(" ")
news_raw['tokenize'] = news_raw['tokenize'].apply(lambda x: [i for i in x if i != "" and i != " "])

news_raw['company_name'] = 0
news_raw['keyword'] = 0 # keybert로 뽑아낸 키워드 컬럼
news_raw['keyword2'] = 0

# list에 담아주기
tokenize_list = news_raw['tokenize']
rep_list = rep_stock['name_a'].str.split(",")

# 맵핑한 데이터 저장하기 위한 csv파일 생성
project_result = pd.DataFrame(columns=['news_id', 'company_word','pk'])

In [2]:
news_raw

,news_id,title,summary,tokenize,company_name,keyword,keyword2
0,345585,Nestle Outshines Rivals With Revenue Growth Le...,nestle sa sailed past ailing consumer goo...,"[nestle, sa, sailed, past, ailing, consumer, g...",0,0,0
1,345586,Total Makes Surprise Profit as Trading Gains O...,total se made a surprise profit after ve...,"[total, se, made, a, surprise, profit, after, ...",0,0,0
2,345590,Airbus Follows Boeing in Paring Output to Weat...,airbus se cut back wide body jet producti...,"[airbus, se, cut, back, wide, body, jet, produ...",0,0,0
3,345591,Virus Uptick Imperils South Europe's Nascent T...,nadine scheiner s efforts to travel from ...,"[nadine, scheiner, s, efforts, to, travel, fro...",0,0,0
4,345595,PG&E Fire Insurance Costs Skyrocket After Bank...,pg&e corp is finding it very costly to b...,"[pg&e, corp, is, finding, it, very, costly, to...",0,0,0
...,...,...,...,...,...,...,...
995,443773,Localiza to Form $9 Billion Car-Rental Giant W...,localiza rent a car sa and unidas two of...,"[localiza, rent, a, car, sa, and, unidas, two,...",0,0,0
996,443778,Mendoza Province Is First to Restructure After...,argentina s mendoza province has reached ...,"[argentina, s, mendoza, province, has, reached...",0,0,0
997,443779,Manufacturing Keeps German Economy on a Recove...,german factories kept europe s biggest ec...,"[german, factories, kept, europe, s, biggest, ...",0,0,0
998,443780,ECB Must Limit Emergency Powers to Temporary C...,the european central bank risks legal tro...,"[the, european, central, bank, risks, legal, t...",0,0,0


## 회사 이름 추출 => 추출 완료 ( company_word.csv 에 저장 )

In [ ]:
# 대표단어 csv파일에서 대표단어가 포함되면 맵핑
i = 0

for token_num, token in enumerate(tokenize_list):  # news_raw data 토큰화
    for rep_num, rep in enumerate(rep_list):  # 대표단어 csv파일에서 대표단어에 해당
        i = i + 1
        if len(rep) == 1:
            if rep[0] in token:
                project_result.loc[i] = [news_raw['news_id'][token_num], rep[0], rep_stock['pk'][rep_num]]

        elif len(rep) == 2:  # 대표단어가 2개로 된 단어일 때
            if rep[0] in token:  # 대표단어의 첫번째 단어가 org단어에 있으면
                found = token.index(rep[0])  # 대표단어의 첫번째 단어와 일치하는 org단어의 인덱스 위치 번호
                try:
                    search = found + 1  # stocklist의 첫번째 단어가 org에 포함됐을때 그 다음 단어
                    search_found = token[search]  # org의 (+1을 한) 다음 단어에 해당
                    if rep[1] == search_found:
                        project_result.loc[i] = [news_raw['news_id'][token_num], rep[0] + " " + rep[1],
                                                 rep_stock['pk'][rep_num]]
                except IndexError:
                    continue

        elif len(rep) == 3:
            if rep[0] in token:
                try:
                    search_found = token[token.index(rep[0]) + 1]
                    if rep[1] == search_found:
                        two_found = token[token.index(rep[0]) + 2]
                        if rep[2] == two_found:
                            project_result.loc[i] = [news_raw['news_id'][token_num],
                                                     rep[0] + " " + rep[1] + " " + rep[2], rep_stock['pk'][rep_num]]
                except IndexError:
                    continue

In [8]:
project_result = pd.merge(project_result, nasdaq_stock, how='left', left_on='pk', right_on='pk')

In [177]:
company_word = pd.read_csv('C:/projectnasdaq/project2/company_word.csv')
company_word

,Unnamed: 0,news_id,company_word,pk,symbol,name
0,0,345585,coca cola,4290.0,KO,Coca-Cola Company (The)
1,1,345585,unilever,7628.0,UL,Unilever PLC
2,2,345585,diageo,2152.0,DEO,Diageo plc
3,3,345585,anheuser busch,1306.0,BUD,Anheuser-Busch Inbev SA Sponsored ADR (Belgium)
4,4,345585,starbucks,6606.0,SBUX,Starbucks Corporation
...,...,...,...,...,...,...
2015,2015,443773,bank of america,860.0,BAC,Bank of America Corporation
2016,2016,443778,first of,2872.0,FLIC,First of Long Island Corporation
2017,2017,443779,ihs,3865.0,IHS,IHS Holding Limited Ordinary Shares
2018,2018,443780,the european,2427.0,EEA,The European Equity Fund Inc


## 키워드 추출 ( Keybert 사용 )=> 추출 완료 ( keyword.csv 에 저장 )

In [ ]:
# Keybert로 키워드 추출 ( 키워드 1-2 자리까지 )
summaries = news_raw['summary']

for sum_num, summary in enumerate(summaries):
    kw_model = KeyBERT()
    keywords = kw_model.extract_keywords(summary)
    news_raw['keyword'][sum_num] = kw_model.extract_keywords(summary, keyphrase_ngram_range=(1, 1))
    news_raw['keyword2'][sum_num] = kw_model.extract_keywords(summary, keyphrase_ngram_range=(1, 2))
#news_raw.to_csv('C:/projectnasdaq/project2/keyword.csv')

## 종목 정보 수집 ( yfinance 패키지 사용 )

In [ ]:
# 종목코드 리스트
stocks = nasdaq_stock['symbol']
# 종목코드에 대한 정보 수집용 데이터프레임 생성
stock_info = pd.DataFrame(columns=['symbol', 'info'])
i = 0

for stock_num, stock in enumerate(stocks):
    tickers = yf.Ticker(stock)
    ticker = tickers.info
    i = i + 1
    #print(stock_num,stock, "===>>",ticker)
    stock_info.loc[i] = [stock, ticker]
    time.sleep(random.uniform(3, 4))

stock_info.to_csv('C:/projectnasdaq/project2/stock_info2.csv')

## 추출된 회사 이름이랑 ( 전처리 한 ) 키워드 연결 ( company_and_keyword.csv 파일에 저장 )

In [ ]:
company_mapping = pd.read_csv('C:/projectnasdaq/project2/company_word.csv')
keyword_mapping = pd.read_csv('C:/projectnasdaq/project2/keyword.csv')

keyword_mapping['keyword'] = keyword_mapping['keyword'].apply(
    lambda x: re.sub('[^a-zA-Z&]', ' ', str(x)).strip())  # 정규식전처리
keyword_mapping['keyword'] = keyword_mapping['keyword'].apply(
    lambda x: re.sub(r"\s+", " ", str(x)).strip())  # 공백 여러개 하나로
keyword_mapping['keyword'] = keyword_mapping['keyword'].str.split(" ")
#keyword_mapping['keyword'].replace(' ',',', regex=True, inplace=True)

keyword_mapping['keyword2'] = keyword_mapping['keyword2'].apply(lambda x: re.sub('[^a-zA-Z&]', ' ', str(x)).strip())
keyword_mapping['keyword2'] = keyword_mapping['keyword2'].str.split("  ")
# 공백으로 토큰화되거나 비어있는 내용으로 토큰화 되어있으면 지우기
keyword_mapping['keyword2'] = keyword_mapping['keyword2'].apply(lambda x: [i for i in x if i != "" and i != " "])

company_mapping[['keyword', 'keyword2']] = 0

keyword_list1 = keyword_mapping['keyword']
keyword_list2 = keyword_mapping['keyword2']

keyword_id = keyword_mapping['news_id']
company_id = company_mapping['news_id']

In [ ]:
for i, j in enumerate(company_id):
    for k, l in enumerate(keyword_id):
        if j == l:
            company_mapping['keyword'][i] = keyword_mapping['keyword'][k]
            company_mapping['keyword2'][i] = keyword_mapping['keyword2'][k]

#company_mapping.to_csv('C:/projectnasdaq/project2/company_and_keyword.csv')

In [95]:
# yahoo_dataset 파일 수정 ( -> symbol name 수정 )
yahoo_dataset = pd.read_csv('C:/projectnasdaq/project2/yahoo_dataset.csv')
yahoo_dataset.insert(1,'new_symbol','0')
yahoo_dataset.insert(2,'name','0')
yahoo_dataset.insert(3,'company_word','0')

In [98]:
rep_stock

,pk,symbol,name,name_a
0,3359.0,GOOG,Alphabet Inc,alphabet
1,21.0,AAPL,Apple Inc,apple
2,4996.0,MSFT,Microsoft Corporation,microsoft
3,3359.0,GOOG,Alphabet Inc,google
4,2746.0,FB,Meta Platforms Inc,facebook
...,...,...,...,...
8260,8157.0,NaN,XPAC Acquisition Corp,[]
8261,8181.0,YCBD^A,cbdMD Inc,[]
8262,8226.0,NaN,FTAC Zeus Acquisition Corp,[]
8263,8242.0,NaN,Zanite Acquisition Corp,[]


In [97]:
yahoo_dataset

,symbol,new_symbol,name,company_word,shortName,longName,quoteType,currency,regularMarketPrice,regularMarketChange,...,averageDailyVolume3Month,marketCap,trailingPE,fiftyTwoWeekLow,fiftyTwoWeekHigh,regularMarketOpen,priceHint,underlyingSymbol,company_info,finance_info
0,A,0,0,0,"Agilent Technologies, Inc.","Agilent Technologies, Inc.",EQUITY,USD,"{'raw': 127.19, 'fmt': '127.19'}","{'raw': 7.330002, 'fmt': '7.33'}",...,"{'raw': 1830575, 'fmt': '1.831M', 'longFmt': '...","{'raw': 37992673280, 'fmt': '37.993B', 'longFm...","{'raw': 30.722223, 'fmt': '30.72'}","{'raw': 112.52, 'fmt': '112.52'}","{'raw': 179.57, 'fmt': '179.57'}","{'raw': 122.12, 'fmt': '122.12'}",2,NaN,"{""address1"": ""5301 Stevens Creek Boulevard"", ""...","{'language': 'en-US', 'region': 'US', 'quoteTy..."
1,AA,0,0,0,Alcoa Corporation,Alcoa Corporation,EQUITY,USD,"{'raw': 45.15, 'fmt': '45.15'}","{'raw': 0.0900002, 'fmt': '0.09'}",...,"{'raw': 6905837, 'fmt': '6.906M', 'longFmt': '...","{'raw': 8327872512, 'fmt': '8.328B', 'longFmt'...","{'raw': 11.43038, 'fmt': '11.43'}","{'raw': 36.49, 'fmt': '36.49'}","{'raw': 98.09, 'fmt': '98.09'}","{'raw': 47.33, 'fmt': '47.33'}",2,NaN,"{""address1"": ""201 Isabella Street"", ""address2""...","{'language': 'en-US', 'region': 'US', 'quoteTy..."
2,AAC,0,0,0,Ares Acquisition Corporation,Ares Acquisition Corporation,EQUITY,USD,"{'raw': 9.86, 'fmt': '9.86'}","{'raw': 0.009999275, 'fmt': '0.01'}",...,"{'raw': 394577, 'fmt': '394,577', 'longFmt': '...","{'raw': 1232499968, 'fmt': '1.232B', 'longFmt'...","{'raw': 89.63636, 'fmt': '89.64'}","{'raw': 9.66, 'fmt': '9.66'}","{'raw': 9.88, 'fmt': '9.88'}","{'raw': 9.85, 'fmt': '9.85'}",2,NaN,"{""address1"": ""245 Park Avenue"", ""address2"": ""4...","{'language': 'en-US', 'region': 'US', 'quoteTy..."
3,AACG,0,0,0,ATA Creativity Global,ATA Creativity Global,EQUITY,USD,"{'raw': 1.62, 'fmt': '1.6200'}","{'raw': 0.24000001, 'fmt': '0.2400'}",...,"{'raw': 12847, 'fmt': '12,847', 'longFmt': '12...","{'raw': 51189732, 'fmt': '51.19M', 'longFmt': ...",NaN,"{'raw': 0.88, 'fmt': '0.8800'}","{'raw': 4.27, 'fmt': '4.2700'}","{'raw': 1.41, 'fmt': '1.4100'}",4,NaN,"{""address1"": ""Building No. 2"", ""address2"": ""Ea...","{'language': 'en-US', 'region': 'US', 'quoteTy..."
4,AACI,0,0,0,Armada Acquisition Corp. I,Armada Acquisition Corp. I,EQUITY,USD,"{'raw': 9.89, 'fmt': '9.89'}","{'raw': 0, 'fmt': '0.00'}",...,"{'raw': 18308, 'fmt': '18,308', 'longFmt': '18...","{'raw': 204816960, 'fmt': '204.817M', 'longFmt...",NaN,"{'raw': 9.34, 'fmt': '9.34'}","{'raw': 9.99, 'fmt': '9.99'}","{'raw': 9.89, 'fmt': '9.89'}",2,NaN,"{""address1"": ""2005 Market Street"", ""address2"":...","{'language': 'en-US', 'region': 'US', 'quoteTy..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9001,ZYME,0,0,0,Zymeworks Inc.,Zymeworks Inc.,EQUITY,USD,"{'raw': 6.33, 'fmt': '6.33'}","{'raw': 0.069999695, 'fmt': '0.07'}",...,"{'raw': 2015414, 'fmt': '2.015M', 'longFmt': '...","{'raw': 365691680, 'fmt': '365.692M', 'longFmt...",NaN,"{'raw': 4.56, 'fmt': '4.56'}","{'raw': 37.19, 'fmt': '37.19'}","{'raw': 6.25, 'fmt': '6.25'}",2,NaN,"{""address1"": ""114 East 4th Avenue"", ""address2""...","{'language': 'en-US', 'region': 'US', 'quoteTy..."
9002,ZYNE,0,0,0,"Zynerba Pharmaceuticals, Inc.","Zynerba Pharmaceuticals, Inc.",EQUITY,USD,"{'raw': 1.26, 'fmt': '1.2600'}","{'raw': 0.110000014, 'fmt': '0.1100'}",...,"{'raw': 522075, 'fmt': '522,075', 'longFmt': '...","{'raw': 54930960, 'fmt': '54.931M', 'longFmt':...",NaN,"{'raw': 0.836, 'fmt': '0.8360'}","{'raw': 4.77, 'fmt': '4.7700'}","{'raw': 1.25, 'fmt': '1.2500'}",4,NaN,"{""address1"": ""80 West Lancaster Avenue"", ""addr...","{'language': 'en-US', 'region': 'US', 'quoteTy..."
9003,ZYXI,0,0,0,"Zynex, Inc.","Zynex, Inc.",EQUITY,USD,"{'raw': 7.62, 'fmt': '7.62'}","{'raw': -0.010000229, 'fmt': '-0.01'}",...,"{'raw': 441919, 'fmt': '441,919', 'longFmt': '...","{'raw': 293369984, 'fmt': '293.37M', 'longFmt'...","{'raw': 15.55102, 'fmt': '15.55'}","{'r

In [106]:
# yahoo_dataset 파일 수정 ( -> symbol name 수정 )
yahoo_dataset_symbol = yahoo_dataset['symbol']
rep_stock_symbol = rep_stock['symbol']

for i, j in enumerate(yahoo_dataset_symbol):
    for k, l in enumerate(rep_stock_symbol):
        if j == l:
            yahoo_dataset['new_symbol'][i] = rep_stock['symbol'][k]
            yahoo_dataset['name'][i] = rep_stock['name'][k]
            yahoo_dataset['company_word'][i] = rep_stock['name_a'][k]
            break

C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD fro

In [108]:
# yahoo_dataset 컬럼 정리
yahoo_dataset.drop(columns=["quoteType", "currency", 'regularMarketPrice', 'regularMarketChange', 'regularMarketChangePercent',
             'regularMarketVolume', 'averageDailyVolume3Month', 'marketCap', 'trailingPE', 'fiftyTwoWeekLow',
             'fiftyTwoWeekHigh', 'regularMarketOpen', 'priceHint', 'underlyingSymbol'], inplace=True)
yahoo_dataset.to_csv('C:/projectnasdaq/project2/yahoo_dataset_mapping.csv')

In [156]:
# company단어들 하나씩만 list에 담기 -> 단어 이름이 같은 종목명들의 키워드 모으기 위해서
company_and_keyword = pd.read_csv('C:/projectnasdaq/project2/company_and_keyword.csv')
company_and_keyword.drop(columns=["Unnamed: 0", 'Unnamed: 0.1'], inplace=True)
company_word = company_and_keyword['company_word']

company_word_same_list = []
for company in company_word:
    if company not in company_word_same_list:
        company_word_same_list.append(company)

company_word_sames = pd.DataFrame(company_word_same_list, columns=['company_word_same'])
company_word_same = company_word_sames['company_word_same']

0                coca cola
1                 unilever
2                   diageo
3           anheuser busch
4                starbucks
              ...         
425              delta air
426        goodrx holdings
427             spi energy
428    petroleo brasileiro
429             braskem sa
Name: company_word_same, Length: 430, dtype: object

In [181]:
# 키워드랑 company_name이랑 데이터프레임에서 맵핑 시키기위한 새로운 dataframe 생성
company__name = pd.DataFrame(columns=['company', 'keyword', 'keyword2'])

# 딕셔너리를 { news_id : keyword } 형태로 만들기 위해서 index를 news_id로 설정
company_and_keyword = company_and_keyword.set_index(keys='news_id', drop=False, inplace=False)

In [182]:
# company__name에 대한 키워드들 리스트로 모아서 column에 넣기
for i, j in enumerate(company_word_same):
    company__name.loc[i] = [j, company_and_keyword[company_and_keyword['company_word'] == j]['keyword'].to_dict(), company_and_keyword[company_and_keyword['company_word'] == j]['keyword2'].to_dict()]

,company,keyword,keyword2
0,coca cola,"{'345585': '['nestle', 'starbucks', 'brands', ...","{'345585': '['nestle sa', 'nestle', 'underperf..."
1,unilever,"{'345585': '['nestle', 'starbucks', 'brands', ...","{'345585': '['nestle sa', 'nestle', 'underperf..."
2,diageo,"{'345585': '['nestle', 'starbucks', 'brands', ...","{'345585': '['nestle sa', 'nestle', 'underperf..."
3,anheuser busch,"{'345585': '['nestle', 'starbucks', 'brands', ...","{'345585': '['nestle sa', 'nestle', 'underperf..."
4,starbucks,"{'345585': '['nestle', 'starbucks', 'brands', ...","{'345585': '['nestle sa', 'nestle', 'underperf..."
...,...,...,...
425,delta air,"{'443844': '['pandemic', 'airline', 'airlines'...","{'443844': '['airline inevitable', 'pandemic c..."
426,goodrx holdings,"{'443702': '['recession', 'stocks', 'stocksthe...","{'443702': '['rampant stock', ' stocks slumped..."
427,spi energy,"{'443710': '['tesla', 'ev', 'spi', 'stocks', '...","{'443710': '['energy stocks', ' spi stock', 'i..."
428,petroleo brasileiro,"{'443728': '['vitol', 'indictment', 'prosecuto...","{'443728': '['bribes vitol', 'vitol firm', 'au..."


In [183]:
# company__name.csv에 column 삽입
company__name.insert(0, 'symbol','0')
company__name.insert(1, 'name','0')
company__name['shortName'] = 0
company__name['longName'] = 0

yahoo_dataset_mapping = pd.read_csv('C:/projectnasdaq/project2/yahoo_dataset_mapping.csv')
company_word = pd.read_csv('C:/projectnasdaq/project2/company_word.csv')

In [192]:
# company__name.csv에 symbol, name 데이터 삽입
company_word_ = company_word['company_word']
company__ = company__name['company']

for i,j in enumerate(company_word_):
    for k, l in enumerate(company__):
        if j==l:
            company__name['symbol'][k] = company_word['symbol'][i]
            company__name['name'][k] = company_word['name'][i]
            break

C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [198]:
# company__name.csv에 yahoo finance 정보 수집 내용 삽입
yahoo_dataset_mapping_symbol = yahoo_dataset_mapping['new_symbol']
company__name_symbol = company__name['symbol']

for i,j in enumerate(yahoo_dataset_mapping_symbol):
    for k,l in enumerate(company__name_symbol):
        if j==l:
            company__name['shortName'][k] = yahoo_dataset_mapping['shortName'][i]
            company__name['longName'][k] = yahoo_dataset_mapping['longName'][i]
            break

C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

In [201]:
# 완성된 news_raw dataset
company__name.to_csv('C:/projectnasdaq/project2/news_raw_dataset.csv')